In [1]:
import os
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import re
import json
import urllib.request
import requests

In [2]:
#https://codeforces.com/contests
#https://codeforces.com/contest/1698/status/A/page/2?order=BY_ARRIVED_DESC

In [3]:
cf_url = 'https://codeforces.com'
C_vers = ['GNU C++14','GNU C++17','GNU C++14','GNU C++20 (64)','MS C++ 2017', 'GNU C++17 (64)','Clang++20 Diagnostics','Clang++17 Diagnostics']
problem_set_url = "https://codeforces.com/api/problemset.problems"

In [4]:
def handle_index(index):
    index_re = '(\d+)_(\w+)'
    result = re.match(index_re, index)
    contest_num = result[1]
    problem_key = result[2]
    return contest_num, problem_key

In [5]:
with open('../_tags.txt','r') as file:
	problems = file.readlines()
problem_index = []
problem_tags = []
for i in range(len(problems)):
	index = list(eval(problems[i]).items())[0][0]
	tags = list(eval(problems[i]).items())[0][1]
	problem_index.append(index)
	problem_tags.append(tags)

In [6]:
drive = webdriver.Chrome(ChromeDriverManager().install())
drive.maximize_window()
wait = WebDriverWait(drive, 10)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/phanvanhung/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.53/chromedriver] found in cache
/var/folders/2r/vgdn5c2j5fb567xb2nm5_qfm0000gn/T/ipykernel_95195/1106099800.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  drive = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
def Scrapping(problem_index):
    contest_num, problem_key = handle_index(problem_index)
    submission_path = '../code_forces/' + str(contest_num) + '_' + str(problem_key)+'/'
    if not os.path.exists(submission_path): os.makedirs(submission_path)
    page_submission = 1
    save_file_num = len(os.listdir(submission_path))
    while save_file_num <= 5:
        file_num = 0
        try:
            drive.get(cf_url + '/contest/' + contest_num + '/status/'+problem_key+'/page/'+ str(page_submission) + "?order=BY_JUDGED_DESC")
        except: break
        xpath_submission_box = '//table//tbody//tr[@data-submission-id]//td'
        submissions = drive.find_elements(By.XPATH, value = xpath_submission_box)
        submission_num = []
        for i in range(len(submissions)):
            if (i % 8 == 0) and submissions[i + 4].text in C_vers:
                submission_num.append(submissions[i].text)
        for i in range(len(submission_num)):
            submission_url = cf_url+'/contest/'+contest_num+'/submission/'+str(submission_num[i])
            try:
                page = requests.get(submission_url)
                page = bs(page.content)
                submission = page.find("pre", attrs={"class":"prettyprint lang-cpp linenums program-source"}).text
            except: pass
            else:
                file_num +=1
                with open(submission_path + str(submission_num[i]) + '.txt', 'w') as file:
                    try:
                        file.write(submission)
                    except: pass
                    file.close()
            if (file_num > 5): break
        if (save_file_num == len(os.listdir(submission_path))): break
        else: save_file_num = len(os.listdir(submission_path))
        page_submission+=1
    page_submission = 1

In [8]:
check = False
for i in range(len(problem_index))[::-1]:
    Scrapping(problem_index[i])